# Veri Ön İşleme
* Veri ön işleme; veri madenciliği modelleri kurulmadan önce veri seti üzerinde yapılan bir takım düzeltme, eksik veriyi tamamlama, tekrarlanan verileri kaldırma, dönüştürme, bütünleştirme, temizleme, normalleştirme, boyut indirgeme vb. işlemlerdir.
* Üç ana başlıkta inceleyebiliriz.

## Veri Temizleme(Data Cleaning)
* Gürültülü Veri(Noisy Data)
* Eksik Veri Analizi(Missing Data Analysis)
* Aykırı Gözlem Analizi(Outlier Analysis)
## Veri Standardizasyonu(Data Standardization)
* 0-1 Dönüşümü (Normalization)
* z-skoruna Dönüştürme(Standardization)
* Logaritmik Dönüşüm(Log Transformation)
## Veri İndirgeme(Data Reduction)
* Gözlem (Observation) Sayısının Azaltılması
* Değişken (Variable) Sayısının Azaltılması

* Bu derste veri temizleme ana başlğı altında yer alan aykırı gözlem analizini ele alacağız.

### Aykırı Gözlem Analizi(Outlier Analysis)
* Aykırı Gözlem : Bir veri setindeki gözlemlerin geri kalanından büyük ölçüde farklı olan gözlemlerdir.
* Diğer bir tanımla genel eğilimin oldukça dışına çıkan gözlemdir.
* Aykırılığı ifade eden nümerik değer aykırı değer,
* Aykırı değeri barından gözlem birimi ise aykırı gözlem olarak tanımlandırılır.

Dosya tüm değişkenleri içerir. Spesifik olarak, medyan ev değeri, orta gelir, konut medyan yaşı, toplam odalar, toplam yatak odaları, nüfus, haneler, enlem ve bu sırayla enlem içerir.

* Veri setimiz; evin değeri(medyan), gelir(medyan), konut yaşı(medyan), toplam oda sayısı, toplam yatak odası sayısı, nüfus, haneler, enlem ve boylam bilgilerini içerir.
* Veri seti hakkında daha detaylı bilgi ve indirmek için:
* https://www.kaggle.com/harrywang/housing

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = sns.load_dataset('diamonds')
df.head()

In [ ]:
df.info()

* Aykırı değer gözlemlemek için veri setimizden `population` sütununu seçelim.

In [ ]:
df_table=df["table"]

* Aykırı gözlemleri görselleştirmek için bir çok teknik var biz burada en yaygın olan kutu grafiği ( Box Plot ) kullanacağız.

In [ ]:
sns.boxplot(x=df_table);

* Grafikten anlaşılacağı üzere üst sınırın dışında aykırı gözlemlerimiz bulunuyor.

## Aykırı Gözlemleri Belirleme(Interquartile Range, IQR Yöntemi)

* Aykırı gözlemleri belirleyebilmemiz için box plotta gördüğünüz gibi sınır değerlerini tanımlamamız gerekiyor. Bunun için IQR tekniğini kullanıyoruz.

* Q1, değişkenin ilk çeyrelik kısmı ( 0.25 )

* Q3, değişkenin üçüncü çeyrekliği oluyor ( 0. 75 )

* Q3-Q1 ile IQR değerini hesaplıyoruz.

* Q1-Q3-IQR tamam şimdi sırada alt sınır ve üst sınır değerlerinde.

* Alt sınır : Q1–1.5*IQR
* Üst sınır : Q3 + 1.5*IQR
* Alt sınırın altındaki ve üst sınırın üstündeki değerler aykırı değer olarak tanımlanacak.

In [ ]:
Q1 = df_table.quantile(0.25)
Q3 = df_table.quantile(0.75)
IQR = Q3-Q1

In [ ]:
IQR

In [ ]:
alt_sinir=Q3-1.5*IQR
alt_sinir

In [ ]:
ust_sinir = Q3 + 1.5*IQR
ust_sinir

* Aykırı değerlerimiz üst sınırdan büyük olan değerlerdir.
* Aşağıdaki kod çıktısı `boolean(True/False)` veri türüdür.

In [ ]:
df_table<alt_sinir

In [ ]:
df_table > ust_sinir

In [ ]:
(df_table < alt_sinir) | (df_table > ust_sinir)

In [ ]:
aykiri=(df_table < alt_sinir)

In [ ]:
df_table[aykiri]#aykırı değerlere erişmek için

In [ ]:
df_table[aykiri].index#aykrı değerlerin indekslerine erişmek için

In [ ]:
aykiri_df=pd.DataFrame(data=df_table[aykiri])#dataframe'ye çevirme
aykiri_df.head()

## Aykırı Gözlemleri Silme

* Silme işlemini gerçekleştirmeden önce `df_population` değişkeninin tipini ve boyutunu öğrenelim.
* Eğer veri türü `DataFrame` değil ise veri türünü `DataFrame`'ye çevirelim.

In [ ]:
type(df_table)

In [ ]:
df_table=pd.DataFrame(data=df_table)

In [ ]:
df_table.shape

* Üst sınırdan büyük olan değerleden yani aykırı değerlerden kurtulmak için yeni bir değişken tanımlayalım.
* Aşağıdaki kodda ~ sembolünün işlevi sağ taraftaki işlemin dışındakileri seçmektir.
* .any(axis = 1) ile de sütun bazında işlem yapmak istediğimizi belirtmiş olduk.

In [ ]:
new_df_table =df_table[~((df_table < (alt_sinir)) | (df_table > (ust_sinir))).any(axis = 1)]
new_df_table.shape

## Ortalama İle Doldurma
* Aykırı değerlerin yerine ortalama değerlerle dolduralım.
* Bunun için `.mean()` parametresini kullanacağız.

In [ ]:
df.table.mean()#ortalama değeri hesaplama

In [ ]:
df_table[aykiri]=df.table.mean()
df_table[aykiri]

## Baskılama Yöntemi
* Aykırı değerler için üst ve alt sınır belirlemiştik.
* Aykırı değerlerimizi baskılama yöntemi ile üst sınıra yakın olanları üst sınıra, alt sınıra yakın olan değerleri alt sınıra eşitliyoruz.
* Üst sınır değerimizi tekrar hesaplayalım.

In [ ]:
Q1 = df_table.quantile(0.25)
Q3 = df_table.quantile(0.75)
IQR = Q3-Q1

In [ ]:
ust_sinir = Q3 + 1.5*IQR
ust_sinir

In [ ]:
alt_sinir=Q3-1.5*IQR
alt_sinir

In [ ]:
aykiri=(df_table < alt_sinir)

In [ ]:
df_table[aykiri]=alt_sinir
df_table[aykiri]

* En son olarak alt sınırda olan aykırı gözlemlerden kurtulduğumuzdan emin olmak için kutu grafiği(Box Plot) ile bir gözlem yapalım.

In [ ]:
sns.boxplot(x=df_table)

# KAYNAKLAR
* https://www.veribilimiokulu.com/buyuk-veri-on-isleme